In [ ]:
# %pip install tqdm

In [41]:
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
import psycopg2
from dotenv import dotenv_values

In [42]:
def get_vacancies_id(header:dict, param:dict=None, URL:str='https://api.hh.ru/vacancies', per_page:int=100, time_dalay:float=0)->json:
    sleep(time_dalay)
    return requests.get(url=URL, headers=header, params=param).json()

In [43]:
JOB_TITLE = 'Аналитик данных'

USER_AGENT = dotenv_values('.env')['USER_AGENT']
BASE_URL='https://api.hh.ru/vacancies'

PER_PAGE = 100

header = {'User-Agent':USER_AGENT}
param = {'text': JOB_TITLE,
          'search_field': 'name',
          'page': 0,
          'per_page': PER_PAGE,
          'only_with_salary': True,
          'locale': 'RU'}

In [44]:
resp = get_vacancies_id(header, param, per_page=PER_PAGE)

VAC_CNT = resp['found']
PAGES_CNT = resp['pages']
current_page = resp['page']
VAC_CNT, PAGES_CNT, current_page

(233, 3, 0)

In [45]:
vac_ids = []
current_page = 0
PAGES_CNT = 999

while current_page <= PAGES_CNT-1:
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']
    PAGES_CNT = resp['pages']

    for item in resp['items']:
        vac_ids.append(item['id'])

    param['page'] = current_page + 1
    resp = get_vacancies_id(header, param, per_page=PER_PAGE)
    current_page = resp['page']


print("It's all OK" if len(set(vac_ids)) == VAC_CNT else f"Smt went WRONG\n{len(set(vac_ids))}___{VAC_CNT}")

It's all OK


In [ ]:
vacancies_df    = pd.DataFrame(columns=['vac_id',
                                        'vac_name',
                                        'city',
                                        'published_at',
                                        'archived',
                                        'expirience'
                                        'alternate_url',
                                        'employer_id',
                                        'employer_name',
                                        'schedule_name',
                                        'employment_name',
                                        'prof_role',
                                        'work_format',
                                        'work_hour',
                                        'work_schedule_by_days'] )

salary_df       = pd.DataFrame(columns=['vac_id',
                                        'salary_from',
                                        'salary_to',
                                        'currency',
                                        'gross',
                                        'range_mode_name',
                                        'frequency'])

requerments_df  = pd.DataFrame(columns=['vac_id',
                                        'response_letter',
                                        'has_test',
                                        'descriptions',
                                        'key_skills'])


for vac_id in tqdm(vac_ids,desc='GET vacancies annotate'):
    vac_resp = get_vacancies_id(header, URL=BASE_URL+'/'+vac_id, time_dalay=0.5)
    

    vacancies_df.loc[len(vacancies_df)] = [vac_resp['id'], \
                                           vac_resp['name'], \
                                           vac_resp['area']['name'], \
                                           vac_resp['published_at'], \
                                           vac_resp['archived'], \
                                           vac_resp['experience']['name'], \
                                           vac_resp['alternate_url'], \
                                           vac_resp['employer']['id'], \
                                           vac_resp['employer']['name'], \
                                           vac_resp['schedule']['name'], \
                                           vac_resp['employment']['name'], \
                                           vac_resp['professional_roles'], \
                                           vac_resp['work_format'], \
                                           vac_resp['working_hours'], \
                                           vac_resp['work_schedule_by_days']]
    
    salary_df.loc[len(salary_df)] = [vac_resp['id'], \
                                     vac_resp['salary']['from'], \
                                     vac_resp['salary']['to'], \
                                     vac_resp['salary']['currency'], \
                                     vac_resp['salary']['gross'], \
                                     vac_resp['salary_range']['mode']['id'], \
                                     vac_resp['salary_range']['frequency']['name']
                                    ]
    
    requerments_df.loc[len(requerments_df)] = [vac_resp['id'], \
                                           vac_resp['response_letter_required'], \
                                           vac_resp['has_test'],
                                           vac_resp['description'],\
                                           vac_resp['key_skills']
                                        ]

In [40]:
vac_resp

{'id': '120585923',
 'premium': False,
 'billing_type': {'id': 'standard', 'name': 'Стандарт'},
 'relations': [],
 'name': 'Аналитик данных',
 'insider_interview': None,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 40000, 'to': None, 'currency': 'RUR', 'gross': False},
 'salary_range': {'from': 40000,
  'to': None,
  'currency': 'RUR',
  'gross': False,
  'mode': {'id': 'MONTH', 'name': 'За\xa0месяц'},
  'frequency': None},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'allow_messages': True,
 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 'schedule': {'id': 'remote', 'name': 'Удаленная работа'},
 'employment': {'id': 'part', 'name': 'Частичная занятость'},
 'department': None,
 'show_contacts': False,
 'contacts': None,
 'description': '<p><strong>KMA.BIZ</strong> (KissMyAds) — одна из крупнейших российских товарных партнерских программ, основана в 2013 году(а